## Modelle nutzen - Lösung

In der letzten Übung haben wir uns einen bestehenden Datensatz (Hauspreise) angeschaut und damit gearbeietet. Heute möchten wir dasselbe mit einem bestehenden Modell zu Erkennung von handschriftlichen Zahlen machen. MNIST ist ein bestehender Datensatz aus handschrifltichen Zahlen in Graustufen mit der Größe 28x28 Pixel. Auf Basis dieses Datensatzes wurde ein Modell trainiert und auf folgender Seite publiziert:

https://www.kaggle.com/models/adhul000/mnist-99 

Wie auch jeder Datensatz haben die meisten publizierten Modelle eine Model Card. Lesen Sie sich die Information auf der Seite zum Modell durch. 

Anhand der Model Card sehen wir, dass wir python 3.x benötigen (das bedeutet irgend ein Python in Version 3, egal ob 3.11, 3.12 oder höher). Weiterhin benötigen wir Tensorflow, eine Bibliothek zum Modell-Training und Modell ausführen (wird in Teil A näher behandelt). Kagglehub und Numpy sollten wir bereits in unserem venv installiert haben, wir können es aber zusätzlich in pip angeben:

In [1]:
pip install kagglehub tensorflow numpy

Note: you may need to restart the kernel to use updated packages.


Als nächstes laden wir uns das Modell von Kaggle herunter. Hierfür nutzen wir, wie bereits beim Datensatz, die Bibliothek kagglehub. Mit Angabe der Umbegungsvariablen ```KAGGLEHUB_CACHE``` können wir bestimmen wo unser Modell lokal auf der Festplatte gespeichert wird:

In [2]:
import kagglehub
import os

# Download latest model version
os.environ['KAGGLEHUB_CACHE'] = 'data'
path = kagglehub.model_download("adhul000/mnist-99/tensorFlow2/default")

print("Path to model files:", path)

/Users/winkelmannsv/git/aki/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to model files: data/models/adhul000/mnist-99/tensorFlow2/default/1


Da wir nun ein fertig trainiertes Modell haben, können wir dies mit Tensorflow laden und verwenden:

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model(os.path.join(path, "best_model.h5"))

Nun benötigen wir noch handschriftliche Zahlen, die wir erkennen können. Schreiben Sie hierfür eine Zahl auf einen Zettel und machen mit Ihrem Smartphone ein Bild davon. Das speichern Sie auf Ihrem Computer, am Besten in diesem Repository im Ordern ```imgs```. Schneiden Sie das Bild mit einem Bildverarbeitungsprogramm so zu, dass die Zahl möglichst das gesamte Bild ausfüllt. Sie finden zwei Beispiele im Ordner: ```3-6.JPG``` und ```3-8.JPG```.

Wie in der Vorlesung behandelt ist nun die Herausforderung das Bild in das richtige Format und die richtige Pixelgröße zu bekommen. Hierfür erstellen wir uns eine Python-Funktion, welche einen Pfad zu einem Bild übergeben bekommt, das Bild lädt, transformiert und schließlich zurück gibt. Dafür setzen wir die Bibliothek PIL ein.

Lesen Sie die Model Card aufmerksam: Welche Schritte müssen Sie ausführen, um Ihr fotografiertes Bild in das richtige Format für das Modell zu bringen? (*Anmerkung: die Schritte können auch eine andere Reihenfolge haben*)

In [10]:
import numpy as np
from PIL import Image

def preprocess_image(image_path: str) -> Image:
    img = Image.open(image_path)  
    img = img.convert("L")  # Schritt 1: Convert to grayscale
    img = img.resize((28, 28))  # Schritt 2: Resize to 28x28
    return img

Nun verwenden wir unsere Funktion, um das Bild in das richtige Format zu transformieren. Mit der Funktion ```show()``` können wir uns das Bild anzeigen lassen:

In [15]:
preprocessed_img = preprocess_image(os.path.join('imgs', '3-6.JPG'))

preprocessed_img.show()
print("Datentyp unseres Bildes: ", type(preprocessed_img))
print("Array Größe des Bildes: ", preprocessed_img.size)

Datentyp unseres Bildes:  <class 'PIL.Image.Image'>
Array Größe des Bildes:  (28, 28)


Nun wird es etwas tricky :). Folgende Schritte müssen wir noch durchführen, damit das Modell unser Bild akzeptiert: 
1) Bisher liegt unser Bild im Datentyp PI.Image.Image vor, das Modell möchte jedoch ein Numpy Array. Führen Sie einen Typecast mit ```np.array(..)``` durch. 
2) Weiterhin sind unsere Graustufen-Bilder im Wertebereich [0, 255], das mag unser Modell gar nicht, sondern möchte Werte im Bereich [0, 1]. Bringen Sie das Bild auf den entsprechenden Wertebereich (wir nennen dies Normalisieren).
3) Das Bild liegt nun als 2-dimensionales Numpy Array (Breite x Höhe) vor. Das Modell erwartet jedoch ein Numpy Array des Shapes (X, 28, 28, 1), d.h. (AnzahlBilder, Breite, Höhe, AnzahlFarbChannels). In unserem Fall haben wir nur ein Bild zum Erkennen und als Grauwert nur einen Kanal, somit ist die neue Shape (1, 28, 28, 1):

In [16]:
np_img = np.array(preprocessed_img)  # Schritt 1: Typecast zu Numpy Array
np_img = np_img / 255.0  # Schritt 2: Normalisieren
np_img = np_img.reshape((1, 28, 28, 1))  # Schritt 3: Reshape

Das war's mit der Vorbereitung, nun lassen wir unser Modell mit ```predict(..)``` das Zeichen erkennen:

In [17]:
predictions = model.predict(np_img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


Für jede Klasse Zahlen (0, 1, 2, .., 9) liefert unser Modell eine Erkennungswahrscheinlichkeit. Mit etwas Glück entspricht die Klasse mit der höchsten Wahrscheinlichkeit unserer Zahl. Ansonsten ist eventuell die Handschrift zu schlecht lesbar oder die Qualität des Bildes (vor allem Kontrast!) zu schlecht. Dann einfach erneut mit einem anderen Bild oder anderer Zahl probieren! 

In [18]:
print(predictions)

[[5.2759154e-03 2.6110390e-03 9.5073623e-04 8.4491272e-04 4.1541308e-03
  3.8685065e-02 8.9909410e-01 1.3661204e-04 4.7277071e-02 9.7051699e-04]]


In [19]:
predicted_class = int(np.argmax(predictions[0]))
print(f"Das Modell erkennt die Zahl als: {predicted_class}")

Das Modell erkennt die Zahl als: 6


Wenn Sie soweit gekommen sind probieren Sie gerne weitere Zahlen aus. Variieren Sie die Schreibweise (gut lesbar, schlecht lesbar) oder die Qualität des Bildes (hell, dunkel, guter Kontrast, schlechter Kontrast) und schauen was das Modell macht.